In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType
import multiprocessing
from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from model_xregopose import xR_egoposemodel
from model_mo2cap2 import mo2cap_ours
from ours_model import Ours_model
from tqdm import tqdm
import os

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main(model_path=None,output_path=None):
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		'/dataset/data/Dataset/TestSet',
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=16,
		num_workers=(multiprocessing.cpu_count() // 3) * 2,
		shuffle=True)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = Ours_model(pretrained_path=model_path).to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		image = test_batch['image'].cuda()
		image_zoom = test_batch['image_zoom'].cuda()
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {
			'image':image,
			# 'image_zoom' : image_zoom,
			}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['pred_pose']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
			# break
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(output_path, res)

	LOGGER.info('Done.')

In [3]:
import glob
path_ =sorted(glob.glob('/workspace/2d_to_3d/apps/exp845'+'/*'))
path_

['/workspace/2d_to_3d/apps/exp845/0.pth',
 '/workspace/2d_to_3d/apps/exp845/1.pth',
 '/workspace/2d_to_3d/apps/exp845/2.pth',
 '/workspace/2d_to_3d/apps/exp845/3.pth',
 '/workspace/2d_to_3d/apps/exp845/4.pth',
 '/workspace/2d_to_3d/apps/exp845/5.pth',
 '/workspace/2d_to_3d/apps/exp845/6.pth',
 '/workspace/2d_to_3d/apps/exp845/7.pth',
 '/workspace/2d_to_3d/apps/exp845/8.pth',
 '/workspace/2d_to_3d/apps/exp845/9.pth',
 '/workspace/2d_to_3d/apps/exp845/best.pth']

In [4]:
import gc
gc.collect()
torch.cuda.empty_cache()
raise

RuntimeError: No active exception to reraise

In [5]:
for i,p in enumerate(path_):
	if i<6:continue
	_path = f'/workspace/2d_to_3d/apps/eval/Ours_new_model845'
	_name = f'baseline845_{i}.json'
	if not os.path.exists(_path):
		os.makedirs(_path)

	main(model_path=p,output_path=os.path.join(_path,_name))

16:46:32 [INFO] Main: Starting demo...
  0%|          | 0/5352 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use Interpolat

{'FullBody': {'All': 71.64915466308594, 'Reacting': 72.27447509765625, 'UpperStretching': 73.57062530517578, 'Gesticuling': 53.55022048950195, 'Walking': 74.62921905517578, 'LowerStretching': 68.96863555908203, 'Talking': 51.1196174621582, 'Patting': 64.11270141601562, 'Gaming': 69.2081527709961, 'Greeting': 54.4398193359375}, 'UpperBody': {'All': 55.070213317871094, 'Reacting': 57.71372985839844, 'UpperStretching': 60.095462799072266, 'Gesticuling': 47.57497024536133, 'Walking': 51.27952194213867, 'LowerStretching': 43.886959075927734, 'Talking': 44.93595504760742, 'Patting': 46.64582443237305, 'Gaming': 48.288818359375, 'Greeting': 46.47333908081055}, 'LowerBody': {'All': 88.22809600830078, 'Reacting': 86.8352279663086, 'UpperStretching': 87.04579162597656, 'Gesticuling': 59.52546310424805, 'Walking': 97.97891998291016, 'LowerStretching': 94.05030822753906, 'Talking': 57.30326461791992, 'Patting': 81.57957458496094, 'Gaming': 90.12748718261719, 'Greeting': 62.40629196166992}}


  7%|▋         | 401/5352 [01:02<11:16,  7.32it/s]

{'FullBody': {'All': 71.80200958251953, 'Reacting': 70.35198211669922, 'UpperStretching': 74.08992767333984, 'Gesticuling': 54.290096282958984, 'Walking': 74.54145812988281, 'LowerStretching': 68.20133972167969, 'Talking': 49.749759674072266, 'Patting': 73.34783935546875, 'Gaming': 67.0553970336914, 'Greeting': 55.4265022277832}, 'UpperBody': {'All': 55.21864318847656, 'Reacting': 56.43246078491211, 'UpperStretching': 60.47470474243164, 'Gesticuling': 47.757293701171875, 'Walking': 51.43403244018555, 'LowerStretching': 44.14405822753906, 'Talking': 44.228275299072266, 'Patting': 51.33330535888672, 'Gaming': 46.82472229003906, 'Greeting': 45.01872634887695}, 'LowerBody': {'All': 88.38536834716797, 'Reacting': 84.27149963378906, 'UpperStretching': 87.70516967773438, 'Gesticuling': 60.822906494140625, 'Walking': 97.64888000488281, 'LowerStretching': 92.25862121582031, 'Talking': 55.2712516784668, 'Patting': 95.36237335205078, 'Gaming': 87.28607940673828, 'Greeting': 65.83429718017578}}


 11%|█         | 601/5352 [01:28<10:01,  7.90it/s]

...

 93%|█████████▎| 5001/5352 [11:12<00:50,  7.00it/s]

{'FullBody': {'All': 67.55704498291016, 'LowerStretching': 64.39391326904297, 'Walking': 66.8009262084961, 'UpperStretching': 69.6868896484375, 'Patting': 79.79156494140625, 'Reacting': 63.214298248291016, 'Gaming': 62.89156723022461, 'Talking': 52.3921012878418, 'Greeting': 57.783164978027344, 'Gesticuling': 55.04612731933594}, 'UpperBody': {'All': 49.59365463256836, 'LowerStretching': 39.228851318359375, 'Walking': 43.1087646484375, 'UpperStretching': 54.21860885620117, 'Patting': 50.80552673339844, 'Reacting': 51.536598205566406, 'Gaming': 45.98296356201172, 'Talking': 41.81958770751953, 'Greeting': 43.16799545288086, 'Gesticuling': 50.16281509399414}, 'LowerBody': {'All': 85.52042388916016, 'LowerStretching': 89.5589828491211, 'Walking': 90.49310302734375, 'UpperStretching': 85.15515899658203, 'Patting': 108.77761840820312, 'Reacting': 74.89200592041016, 'Gaming': 79.80016326904297, 'Talking': 62.96461868286133, 'Greeting': 72.39833068847656, 'Gesticuling': 59.92943572998047}}


 97%|█████████▋| 5201/5352 [11:39<00:21,  6.91it/s]

{'FullBody': {'All': 67.58399963378906, 'LowerStretching': 64.40831756591797, 'Walking': 66.8259506225586, 'UpperStretching': 69.72362518310547, 'Patting': 79.49126434326172, 'Reacting': 63.28596496582031, 'Gaming': 62.938053131103516, 'Talking': 52.377811431884766, 'Greeting': 57.62351608276367, 'Gesticuling': 55.1271858215332}, 'UpperBody': {'All': 49.613040924072266, 'LowerStretching': 39.24311065673828, 'Walking': 43.18696975708008, 'UpperStretching': 54.23309326171875, 'Patting': 50.57011413574219, 'Reacting': 51.57201385498047, 'Gaming': 46.013431549072266, 'Talking': 41.797122955322266, 'Greeting': 43.11880874633789, 'Gesticuling': 50.161739349365234}, 'LowerBody': {'All': 85.55494689941406, 'LowerStretching': 89.57352447509766, 'Walking': 90.46493530273438, 'UpperStretching': 85.21415710449219, 'Patting': 108.41241455078125, 'Reacting': 74.99993133544922, 'Gaming': 79.8626708984375, 'Talking': 62.958492279052734, 'Greeting': 72.12822723388672, 'Gesticuling': 60.092628479003906}

100%|██████████| 5352/5352 [11:59<00:00,  7.44it/s]
17:46:40 [INFO] Main: Saving evaluation results...
17:46:40 [INFO] Main: Done.
